In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, "../")

In [ ]:
from src.model.deeplabv3 import TrashDetector
from src.data.dataset import TrashDataset
from torch.utils.data import DataLoader

In [ ]:
from torchvision import transforms
import torch
import pytorch_lightning as pl

In [ ]:
val_size = 0.2
batch_size = 2
CONFIG = {
    "img_size": 224,
    "batch_size": batch_size,
    "val_size": val_size,
}

# DataLoader

In [ ]:
transform = preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((CONFIG["img_size"], CONFIG["img_size"])),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
dataset = TrashDataset("../data/raw/train-all.csv", "../data/raw/train", transform=transform)

In [ ]:
train_set, val_set = torch.utils.data.random_split(
    dataset, 
    [
        len(dataset) - int(len(dataset) * val_size) , 
        int(len(dataset) * val_size) 
    ]
)

In [ ]:
train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=8)
val_dataloader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=8)

# Model

In [ ]:
model = TrashDetector()

In [ ]:
trainer = pl.Trainer()

In [ ]:
trainer.fit(model, train_dataloader, val_dataloader)

In [ ]:
model